In [1]:
from functions import *
from torch.optim import Adam
from functions_pt import *

In [2]:
embed_size = 128
dropout = 0.1
multiple_factor = 1
layers = 2
batch_size = 64
agg = 'mean' # mean,sum,min,max
model = pointNet(layers,embed_size,agg,dropout,multiple_factor)#.to('cuda')
#model.load_state_dict(torch.load('baseline_noBN.pt'))
#model.eval()
model = model.to('cuda')

In [3]:
words_embed = np.load('possible_words_embed.npy')

RL Loop

In [4]:
# model trained from start
lr = 6e-5
clip = 0.03
epochs = 3
damp = 1
iterations = 30
max_data = 2000
states = set()
opt = Adam(model.parameters(),betas=(0.85, 0.95),lr=lr)
for i in range(iterations):
    # policy evaluation
    model.eval()
    score = evaluate_save(matrix,np.arange(2309),policy_model,2309,0,model=model,words_embed=words_embed,top=0.6,
                         eps=0.09/(i+1))
    print('iter:{}, score:{}'.format(i,score))
    
    # prepare data for training
    index,val,log_prob = zip(*out)
    for element in index:
        states.add(tuple(element))
    print('set size:{}'.format(len(states)))
    prob = np.array(log_prob)
    prob -= prob.mean()
    prob = np.exp(prob/damp)
    prob = torch.tensor(prob,dtype=torch.float32,device='cuda')
    length = torch.tensor([len(i) for i in index],dtype=torch.float32,device='cuda')
    words = torch.tensor(words_embed[np.concatenate(index)]).long().to('cuda')
    ys = torch.tensor(val,dtype=torch.float32,device='cuda')

    
    # policy update
    model.train()
    for _ in range(epochs):
        yhat = model((words,length))
        loss = torch.sum(prob*(ys-yhat)**2)/torch.sum(prob)
        loss.backward()
        print('loss:{}'.format(loss.item()))
        clip_grad_value_(model.parameters(),clip)
        opt.step()
        opt.zero_grad()
    #print(model.w)
    while len(out)>max_data:
        out.pop(0)

iter:0, score:3.7167605023819843
set size:410
loss:0.7993103265762329
loss:0.6610571146011353
loss:0.537652850151062
iter:1, score:3.6383715894326554
set size:773
loss:0.37487322092056274
loss:0.3575732409954071
loss:0.42171216011047363
iter:2, score:3.6245127760935483
set size:1110
loss:0.1301199495792389
loss:0.1377747803926468
loss:0.2738572657108307
iter:3, score:3.5201385881333924
set size:1392
loss:0.10861077159643173
loss:0.262614905834198
loss:0.2393147498369217
iter:4, score:3.541359896058901
set size:1661
loss:0.28312626481056213
loss:0.13119876384735107
loss:0.2226516157388687
iter:5, score:3.521870939800779
set size:1927
loss:0.16784915328025818
loss:0.08862055838108063
loss:0.17878347635269165
iter:6, score:3.5495885664789983
set size:2206
loss:0.12500956654548645
loss:0.16198591887950897
loss:0.2519560158252716
iter:7, score:3.505413598960588
set size:2483
loss:0.07312078773975372
loss:0.1162087544798851
loss:0.13688907027244568
iter:8, score:3.5313988739714173
set size:2

In [5]:
torch.save(model.state_dict(), 'RL0.pt')

In [6]:
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6,
                         eps=0)

3.437418796015592

In [4]:
# model trained from start
# eps greedy
lr = 6e-5
clip = 0.03
epochs = 3
damp = 1
iterations = 30
max_data = 2000
states = set()
opt = Adam(model.parameters(),betas=(0.85, 0.95),lr=lr)
for i in range(iterations):
    # policy evaluation
    model.eval()
    score = evaluate_save(matrix,np.arange(2309),policy_model_eps,2309,0,model=model,words_embed=words_embed,top=0.6,
                         eps=0.25/(i+1)**(0.5))
    print('iter:{}, score:{}'.format(i,score))
    
    # prepare data for training
    index,val,log_prob = zip(*out)
    for element in index:
        states.add(tuple(element))
    print('set size:{}'.format(len(states)))
    prob = np.array(log_prob)
    prob -= prob.mean()
    prob = np.exp(prob/damp)
    prob = torch.tensor(prob,dtype=torch.float32,device='cuda')
    length = torch.tensor([len(i) for i in index],dtype=torch.float32,device='cuda')
    words = torch.tensor(words_embed[np.concatenate(index)]).long().to('cuda')
    ys = torch.tensor(val,dtype=torch.float32,device='cuda')

    
    # policy update
    model.train()
    for _ in range(epochs):
        yhat = model((words,length))
        loss = torch.sum(prob*(ys-yhat)**2)/torch.sum(prob)
        loss.backward()
        print('loss:{}'.format(loss.item()))
        clip_grad_value_(model.parameters(),clip)
        opt.step()
        opt.zero_grad()
    #print(model.w)
    while len(out)>max_data:
        out.pop(0)

iter:0, score:3.5595495885664814
set size:400
loss:0.7726379632949829
loss:0.7377428412437439
loss:0.6685836315155029
iter:1, score:3.5188393243828506
set size:733
loss:0.49167317152023315
loss:0.374877393245697
loss:0.2742391526699066
iter:2, score:3.4876569943698588
set size:1034
loss:0.19567330181598663
loss:0.1443445384502411
loss:0.08508361130952835
iter:3, score:3.4595062797747964
set size:1122
loss:0.08880160003900528
loss:0.09266699850559235
loss:0.21844609081745148
iter:4, score:3.49935036812473
set size:1400
loss:0.12617264688014984
loss:0.11462590098381042
loss:0.10650429129600525
iter:5, score:3.4512776093547
set size:1473
loss:0.0920286774635315
loss:0.16089190542697906
loss:0.07193559408187866
iter:6, score:3.4491121697704634
set size:1523
loss:0.06548918038606644
loss:0.060486335307359695
loss:0.10920626670122147
iter:7, score:3.4564746643568647
set size:1579
loss:0.15068767964839935
loss:0.06904400885105133
loss:0.116324283182621
iter:8, score:3.448679081853617
set size

In [5]:
torch.save(model.state_dict(), 'RL_eps.pt')

In [5]:
evaluate(matrix,np.arange(2309),policy_model_eps,2309,model=model,words_embed=words_embed,top=0.6,
                         eps=0)

3.5889995669120847

In [2]:
max_ = -1
for k,v in mapping.items():
    if v>max_:
        k_max = k
        max_ = v

In [4]:
evaluate(matrix[:,np.array(k_max)],np.array(k_max),policy_lookup,len(k_max))

2.1029411764705874

In [4]:
max_

2.1029411764705874

In [8]:
# slightly worse than # of steps required by mapping as expected
evaluate(matrix[:,np.array(k_max)],np.array(k_max),policy_model,len(k_max),model=model,words_embed=words_embed,top=0.6)

2.1249999999999987

In [2]:
# entropy_best policy
    steps = evaluate(matrix,np.arange(2309),policy_entropy_best,2309,factor=i)
    print('factor:{}, steps:{}'.format(i,steps))

factor:0.0, steps:3.465569510610656
factor:0.5, steps:3.4638371589432673
factor:1.0, steps:3.4634040710264204
factor:1.5, steps:3.4634040710264204
factor:2.0, steps:3.4634040710264204
factor:2.5, steps:3.462970983109573
factor:3.0, steps:3.4638371589432673
factor:3.5, steps:3.4642702468601136


KeyboardInterrupt: 

In [2]:
# entropy policy
for i in np.linspace(0,7,15):
    steps = evaluate(matrix,np.arange(2309),policy_entropy,2309,factor=i)
    print('factor:{}, steps:{}'.format(i,steps))

factor:0.0, steps:3.5275010827197937
factor:0.5, steps:3.4638371589432673
factor:1.0, steps:3.4634040710264204
factor:1.5, steps:3.4634040710264204
factor:2.0, steps:3.4634040710264204
factor:2.5, steps:3.462970983109573
factor:3.0, steps:3.4638371589432673
factor:3.5, steps:3.4642702468601136


KeyboardInterrupt: 

In [2]:
# entropy policy with depth info
for i in np.linspace(1,2,5):
    steps = evaluate_depth(matrix,np.arange(2309),policy_entropy_depth,2309,factor=i,depth=1)
    print('factor:{}, steps:{}'.format(i,steps))

factor:1.0, steps:3.4634040710264204
factor:1.25, steps:3.462970983109573
factor:1.5, steps:3.4638371589432673
factor:1.75, steps:3.4642702468601136
factor:2.0, steps:3.4655695106106554


In [5]:
# see how well model does on full problem that it is not trained on
#     if count == 1: return 1
#     if count == 2: return 1.5
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.5266349068860987

In [6]:
# if entro == best:return row
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.8778692074491117

In [5]:
#                 if c == 1:
#                     continue
#                 if c == 2:
#                     value += p
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.6223473365093115

In [5]:
# old policy_model
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.558683412732789

In [5]:
# new policy_model
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.7756604590731935

In [7]:
# new + if entro==best: return row
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.873538328280638

In [10]:
# new policy_model +
#                 if c == 1:
#                     continue
#                 if c == 2:
#                     value += p
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.5240363793850142